#### `sparql_dataframe` do not provide the possibilty to set the userAgent. Therefore, the file `queryWikidata` slightly modifies the code of the library in order to set the userAgent, required by Wikidata in order to send requests t its sparql endpoint

In [1]:
#import sparql_dataframe
from pandas import *
from queryWikidata import query_wikidata

In [2]:
endpoint = "https://query.wikidata.org/sparql"
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:105.0) Gecko/20100101 Firefox/105.0"

## The query used for the paper

In [3]:
paper_query = """
SELECT DISTINCT ?writer ?writerLabel ?imdb
WHERE {
  VALUES ?citizen {wd:Q38 wd:Q172579}
  VALUES ?role {wd:Q6625963 wd:Q49757}
  ?writer wdt:P345 ?imdb ;
          rdfs:label ?writerLabel ;
          (wdt:P106 | p:P106 / ps:P106) ?role ;
          (wdt:P27 | p:P27 / ps:P27) ?citizen.
  FILTER (lang(?writerLabel) = 'en') 
}
GROUP BY ?writer ?writerLabel ?imdb 
"""

In [4]:
paper_results = query_wikidata(endpoint, paper_query, user_agent, post=True).fillna("")
paper_results

,writer,writerLabel,imdb
0,http://www.wikidata.org/entity/Q193018,Gianni Rodari,nm0734427
1,http://www.wikidata.org/entity/Q3751286,Franco Enna,nm0257850
2,http://www.wikidata.org/entity/Q3767161,Giovanni Drovetti,nm0238144
3,http://www.wikidata.org/entity/Q527124,Vincenzo De Crescenzo,nm1876866
4,http://www.wikidata.org/entity/Q3791865,Idolina Landolfi,nm2285763
...,...,...,...
227,http://www.wikidata.org/entity/Q1132008,Goffredo Parise,nm0661650
228,http://www.wikidata.org/entity/Q451312,Umberto Saba,nm0754460
229,http://www.wikidata.org/entity/Q2022,Cesare Pavese,nm0667610
230,http://www.wikidata.org/entity/Q2895923,Beniamino Joppolo,nm0429778


____________________


## `writers_query`
It retrieves every writer, prosaist, novelist or poet that has the Italian citizenship or lived in a country or historical country whose capital city is now an Italian city (to include also writers living, for example, in the Republic of Florence).
Writers are novelists, poets, prosaists or common "writers". However, when an entity is also a screenwriter, it has to be at least also a novelist or a poet. We restrict the field to authors born after 1200

Problems:
1. Is it better to restrict the threshold of being at least a poet or a novelist as well as a screenwriter excluding the poet possibility (an author to be included should be a novelist if he/she is a screewriter)?

In [5]:
wq_post_1200 = """
SELECT DISTINCT ?writer ?writerLabel ?imdb
WHERE {
  #get writers having an IMDb ID and a Date of Born
  ?writer wdt:P345 ?imdb ;
          wdt:P569 ?dob .
  {
    {
      ?historicalCountry wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?historicalCountry wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?historicalCountry . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118} #modern/contemporary Italy can be one of these entities
    ?writer (wdt:P27 | p:P27 / ps:P27) ?mcItaly . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118} #modern/contemporary Italy can be one of these entities
    ?writer wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P17 ?mcItaly .
  }

  {
    VALUES ?otherRole {wd:Q6625963 wd:Q49757}   #get writers that, if screenwriters, are also either novelists or poets
    ?writer wdt:P106 wd:Q28389 ;
           wdt:P106 ?otherRole.
  }UNION{
    VALUES ?role {wd:Q6625963 wd:Q49757 wd:Q36180 wd:Q12144794}  #get writers that are novelists, poets, prosaists, common wikidata "writers" but NOT screenwriters
    ?writer (wdt:P106 | p:P106 / ps:P106) ?role .
    FILTER NOT EXISTS {?writer wdt:P106 wd:Q28389}
  }
  
  FILTER ("1200-01-01"^^xsd:dateTime <= ?dob) 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

GROUP BY ?writer ?writerLabel ?imdb
"""

In [6]:
writers_post_1200_df = query_wikidata(endpoint, wq_post_1200, user_agent, True).fillna("")
writers_post_1200_df

,writer,writerLabel,imdb
0,http://www.wikidata.org/entity/Q3851252,Massimo Fini,nm1765100
1,http://www.wikidata.org/entity/Q219491,Arrigo Boito,nm0092435
2,http://www.wikidata.org/entity/Q3858037,Milo de Angelis,nm5468653
3,http://www.wikidata.org/entity/Q290287,Katie Boyle,nm0102207
4,http://www.wikidata.org/entity/Q3991530,Tina Lagostena Bassi,nm0481385
...,...,...,...
718,http://www.wikidata.org/entity/Q505383,Rafael Sánchez Ferlosio,nm0844710
719,http://www.wikidata.org/entity/Q650303,Riccardo Bacchelli,nm0995911
720,http://www.wikidata.org/entity/Q33528673,Fabio Genovesi,nm2314156
721,http://www.wikidata.org/entity/Q2248723,Roberto Vecchioni,nm3896285


## Query for work connections in wikidata

The first query in the first block uses only the IDs of the audiovisual works to retrieve data.

The second uses also the IDs of the writers

In [7]:
work_connections_query = """ 
SELECT DISTINCT ?audioVisualWork ?audioVisualWorkLabel ?imdbAudioVisual
WHERE {
  {
    {
      ?historicalCountry wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?historicalCountry wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?historicalCountry . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer (wdt:P27 | p:P27 / ps:P27) ?mcItaly . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P17 ?mcItaly .
  }

  {
    VALUES ?otherRole {wd:Q6625963 wd:Q49757}   #get writers that, if screenwriters, are also either novelists or poets
    ?writer wdt:P106 wd:Q28389 ;
           wdt:P106 ?otherRole.
  }UNION{
    VALUES ?role {wd:Q6625963 wd:Q49757 wd:Q36180 wd:Q12144794}  #get writers that are novelists, poets, prosaists, common wikidata "writers" but NOT screenwriters
    ?writer (wdt:P106 | p:P106 / ps:P106) ?role .
    FILTER NOT EXISTS {?writer wdt:P106 wd:Q28389}
  }
  
  
  
  VALUES ?workType {wd:Q7725634 wd:Q47461344 wd:Q571 wd:Q3331189}   #define what are the possible types of writtenWrok
  ?writtenWork wdt:P50 ?writer ;                #get written wroks written by our writers having the types defined above
               wdt:P31 ?workType .
  
  ?audioVisualWork wdt:P345 ?imdbAudioVisual .  #define audioVisual work as everyting having an imdb ID y now (so to grasp as many products as possile as first)
  {                                             #get aV works derived from the works written by our writers, or inspired by them
    VALUES ?derivedBy {wdt:P737 wdt:P144 wdt:P8371}
    ?audioVisualWork ?derivedBy ?writtenWork .    
  }UNION{
    ?writtenWork wdt:P4969 ?audioVisualWork .    
  }
  
  
  
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

GROUP BY ?audioVisualWork ?audioVisualWorkLabel ?imdbAudioVisual
"""

In [8]:
work_connections_query_writers_id = """
SELECT DISTINCT ?audioVisualWork ?audioVisualWorkLabel  ?imdbAudioVisual
WHERE {
  #get writers having an IMDb ID
  ?writer wdt:P345 ?imdb .
  {
    {
      ?historicalCountry wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?historicalCountry wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?historicalCountry . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer (wdt:P27 | p:P27 / ps:P27) ?mcItaly . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P17 ?mcItaly .
  }

  {
    VALUES ?otherRole {wd:Q6625963 wd:Q49757}   #get writers that, if screenwriters, are also either novelists or poets
    ?writer wdt:P106 wd:Q28389 ;
           wdt:P106 ?otherRole.
  }UNION{
    VALUES ?role {wd:Q6625963 wd:Q49757 wd:Q36180 wd:Q12144794}  #get writers that are novelists, poets, prosaists, common wikidata "writers" but NOT screenwriters
    ?writer (wdt:P106 | p:P106 / ps:P106) ?role .
    FILTER NOT EXISTS {?writer wdt:P106 wd:Q28389}
  }
  
  
  
  VALUES ?workType {wd:Q7725634 wd:Q47461344}   #define what are the possible types of writtenWrok
  ?writtenWork wdt:P50 ?writer ;                #get written wroks written by our writers having the types defined above
               wdt:P31 ?workType .
  
  ?audioVisualWork wdt:P345 ?imdbAudioVisual .  #define audioVisual work as everyting having an imdb ID y now (so to grasp as many products as possile as first)
  {                                             #get aV works derived from the works written by our writers, or inspired by them
    VALUES ?derivedBy {wdt:P737 wdt:P144 wdt:P8371}
    ?audioVisualWork ?derivedBy ?writtenWork .    
  }UNION{
    ?writtenWork wdt:P4969 ?audioVisualWork .    
  }
  
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

GROUP BY ?audioVisualWork ?audioVisualWorkLabel ?imdbAudioVisual
"""

In [9]:
work_connections_df = query_wikidata(endpoint, work_connections_query, user_agent, post= True).fillna("")
work_connections_df

,audioVisualWork,audioVisualWorkLabel,imdbAudioVisual
0,http://www.wikidata.org/entity/Q967661,Pinocchio 3000,tt0332318
1,http://www.wikidata.org/entity/Q1425815,Captain Blood,tt0026174
2,http://www.wikidata.org/entity/Q18429,Fellini's Casanova,tt0074291
3,http://www.wikidata.org/entity/Q834460,The End Is My Beginning,tt1529292
4,http://www.wikidata.org/entity/Q683014,The Garden of the Finzi-Continis,tt0065777
...,...,...,...
154,http://www.wikidata.org/entity/Q5638667,Pinocchio,tt0818692
155,http://www.wikidata.org/entity/Q31185402,Filomena Marturano,tt0471700
156,http://www.wikidata.org/entity/Q1130910,Dante's Inferno,tt0026262
157,http://www.wikidata.org/entity/Q3791184,I promessi sposi,tt0172043


In [10]:
work_connections_query_wid_df = query_wikidata(endpoint, work_connections_query_writers_id, user_agent, post= True).fillna("")
work_connections_query_wid_df

,audioVisualWork,audioVisualWorkLabel,imdbAudioVisual
0,http://www.wikidata.org/entity/Q114725240,The Hummingbird,tt14589916
1,http://www.wikidata.org/entity/Q1077585,The Truce,tt0117959
2,http://www.wikidata.org/entity/Q3760834,Geppetto,tt0200656
3,http://www.wikidata.org/entity/Q28214950,La fameuse invasion des ours en Sicile,tt6284064
4,http://www.wikidata.org/entity/Q48967415,The Name of the Rose,tt7572868
...,...,...,...
132,http://www.wikidata.org/entity/Q3792653,Il Decamerone,tt0167877
133,http://www.wikidata.org/entity/Q18206120,Corruption in the Palace of Justice,tt4103296
134,http://www.wikidata.org/entity/Q3665312,Caesar the Conqueror,tt0057105
135,http://www.wikidata.org/entity/Q1024861,Cabiria,tt0003740


In [11]:
concat_dfs = concat([work_connections_df, work_connections_query_wid_df])
concat_dfs.drop_duplicates()
concat_dfs

,audioVisualWork,audioVisualWorkLabel,imdbAudioVisual
0,http://www.wikidata.org/entity/Q967661,Pinocchio 3000,tt0332318
1,http://www.wikidata.org/entity/Q1425815,Captain Blood,tt0026174
2,http://www.wikidata.org/entity/Q18429,Fellini's Casanova,tt0074291
3,http://www.wikidata.org/entity/Q834460,The End Is My Beginning,tt1529292
4,http://www.wikidata.org/entity/Q683014,The Garden of the Finzi-Continis,tt0065777
...,...,...,...
132,http://www.wikidata.org/entity/Q3792653,Il Decamerone,tt0167877
133,http://www.wikidata.org/entity/Q18206120,Corruption in the Palace of Justice,tt4103296
134,http://www.wikidata.org/entity/Q3665312,Caesar the Conqueror,tt0057105
135,http://www.wikidata.org/entity/Q1024861,Cabiria,tt0003740


#### They do not include different works!

_______________________


# Queries by progressively filtering results

#### `qf_0` retrieves 103805 entities

In [28]:
qf_0 = """
SELECT DISTINCT ?writer ?writerLabel 
WHERE {
    {
      ?country wdt:P706 wd:Q145694 . #country is anything located in the Italian peninsula
    } UNION {
      ?country wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }UNION{
      VALUES ?country {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118}
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?country .    
                      
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
df0 = query_wikidata(endpoint, qf_0, user_agent, True).fillna("")
df0


,writer,writerLabel
0,http://www.wikidata.org/entity/Q82919,Alessandro Rossi
1,http://www.wikidata.org/entity/Q167075,Valeria Ciavatta
2,http://www.wikidata.org/entity/Q542939,Maria Lea Pedini-Angelini
3,http://www.wikidata.org/entity/Q557019,Marino Riccardi
4,http://www.wikidata.org/entity/Q581258,Bartolomeo Borghesi
...,...,...
106149,\tat org.eclipse.jetty.util.thread.strategy.Ea...,
106150,\tat org.eclipse.jetty.util.thread.ReservedThr...,
106151,\tat org.eclipse.jetty.util.thread.QueuedThrea...,
106152,\tat org.eclipse.jetty.util.thread.QueuedThrea...,


#### `qf_1` adds the possible occupation of the writers
- results: 9788         

In [16]:
qf_1 = """
SELECT DISTINCT ?writer ?writerLabel 
WHERE {
    {
      ?country wdt:P706 wd:Q145694 . #country is anything located in the Italian peninsula
    } UNION {
      ?country wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }UNION{
      VALUES ?country{wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118}
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?country .   
            
    VALUES ?occupation {wd:Q36180 wd:Q6625963 wd:Q49757 wd:Q12144794}
    ?writer wdt:P106 ?occupation .

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
df1 = query_wikidata(endpoint, qf_1, user_agent, True).fillna("")
df1

,writer,writerLabel
0,http://www.wikidata.org/entity/Q120553,Paul Videsott
1,http://www.wikidata.org/entity/Q153700,Oriana Fallaci
2,http://www.wikidata.org/entity/Q161933,Alberto Moravia
3,http://www.wikidata.org/entity/Q165351,Mario Toso
4,http://www.wikidata.org/entity/Q175778,Corrado Balducci
...,...,...
9785,http://www.wikidata.org/entity/Q112134947,Antonino Traina
9786,http://www.wikidata.org/entity/Q337311,Antonio Rosmini-Serbati
9787,http://www.wikidata.org/entity/Q45189682,Giovanni Battista Giovio
9788,http://www.wikidata.org/entity/Q64685954,Girolamo Volpe


#### `qf_2` adds a filter to retrieve only writers with a link to a work in wikidata
- results: 1632

In [17]:
qf_2 = """
SELECT DISTINCT ?writer ?writerLabel 
WHERE {
    {
      ?country wdt:P706 wd:Q145694 . #country is anything located in the Italian peninsula
    } UNION {
      ?country wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }UNION{
      VALUES ?country{wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118}
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?country .

    VALUES ?occupation {wd:Q36180 wd:Q6625963 wd:Q49757 wd:Q12144794}
    ?writer wdt:P106 ?occupation .                                      


    { ?something wdt:P170 ?writer .               
     }UNION{
      ?something wdt:P50 ?writer .
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}
      ?writer ?authorOf ?something .
    }                           

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

df2 = query_wikidata(endpoint, qf_2, user_agent, True).fillna("")
df2

,writer,writerLabel
0,http://www.wikidata.org/entity/Q2022,Cesare Pavese
1,http://www.wikidata.org/entity/Q765,Dario Fo
2,http://www.wikidata.org/entity/Q13888,Luchino Visconti
3,http://www.wikidata.org/entity/Q12807,Umberto Eco
4,http://www.wikidata.org/entity/Q12085,Gianluigi Bonelli
...,...,...
1628,http://www.wikidata.org/entity/Q112259235,Francesco Giorgi
1629,http://www.wikidata.org/entity/Q113633980,José Ceppi
1630,http://www.wikidata.org/entity/Q98834184,Giuseppe Mazzini
1631,http://www.wikidata.org/entity/Q3907875,Pompeo Litta Biumi


#### `qf_3` adds a filter to retrieve only writers with at least a link to a work on Wikidata that have also an IMDb ID
- results: 420

In [18]:
qf_3 = """
SELECT DISTINCT ?writer ?writerLabel 
WHERE {
    {
      ?country wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?country wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }UNION{
      VALUES ?country{wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118}
    }
  ?writer (wdt:P27 | p:P27 / ps:P27) ?country .

  VALUES ?occupation {wd:Q36180 wd:Q6625963 wd:Q49757 wd:Q12144794}
  ?writer wdt:P106 ?occupation .                                      
  
  
  { ?something wdt:P170 ?writer .               
  }UNION{
  ?something wdt:P50 ?writer .
  }UNION{
  VALUES ?authorOf {wdt:P1455 wdt:P800}
  ?writer ?authorOf ?work .
  }                                                                    
  
  ?writer wdt:P345 ?imdbId .                                                                        
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

df3 = query_wikidata(endpoint, qf_3, user_agent, True).fillna("")
df3

,writer,writerLabel
0,http://www.wikidata.org/entity/Q2022,Cesare Pavese
1,http://www.wikidata.org/entity/Q765,Dario Fo
2,http://www.wikidata.org/entity/Q2291,Totò
3,http://www.wikidata.org/entity/Q12807,Umberto Eco
4,http://www.wikidata.org/entity/Q13888,Luchino Visconti
...,...,...
415,http://www.wikidata.org/entity/Q3750263,Francesco Mastriani
416,http://www.wikidata.org/entity/Q3763690,Giannino Antona Traversi
417,http://www.wikidata.org/entity/Q3847140,Maria Antonietta Torriani
418,http://www.wikidata.org/entity/Q12786354,Branka Jurca


#### `qf_4` maintains retrieves all the writers that have an ID on IMDb without looking for a link to a work in Wikidata
- results: 1169

In [19]:
qf_4 = """
SELECT DISTINCT ?writer ?writerLabel 
WHERE {
    {
      ?country wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?country wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }UNION{
      VALUES ?country{wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118}
    }
  ?writer (wdt:P27 | p:P27 / ps:P27) ?country .

  VALUES ?occupation {wd:Q36180 wd:Q6625963 wd:Q49757 wd:Q12144794}
  ?writer wdt:P106 ?occupation .                                      
                                                 
  
  ?writer wdt:P345 ?imdbId .                                                                        
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

df4 = query_wikidata(endpoint, qf_4, user_agent, True).fillna("")
df4

,writer,writerLabel
0,http://www.wikidata.org/entity/Q153700,Oriana Fallaci
1,http://www.wikidata.org/entity/Q161933,Alberto Moravia
2,http://www.wikidata.org/entity/Q213482,Giuseppe Tomasi di Lampedusa
3,http://www.wikidata.org/entity/Q268443,Suso Cecchi d'Amico
4,http://www.wikidata.org/entity/Q275979,Elsa Morante
...,...,...
1164,http://www.wikidata.org/entity/Q3758767,Gastone Monaldi
1165,http://www.wikidata.org/entity/Q3769244,Giulia Cassini Rizzotto
1166,http://www.wikidata.org/entity/Q3846392,Marco Praga
1167,http://www.wikidata.org/entity/Q63257348,Giuseppe Petrai


#### `qf_5` aims at removing the first filters in order to retrieve all the italians that have linked work on wikipidia regardless of their occupation
- results: 2368

### PROBLEMI NEI RISULTATI: PERCHè PRENDE CALCIATORI E COS ASONO GLI ULTIMI RISULTATI???

In [35]:
qf_5 = """
SELECT DISTINCT ?writer ?writerLabel 
WHERE {
    {
      ?country wdt:P706 wd:Q145694 . #country is anything located in the Italian peninsula
    } UNION {
      ?country wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }UNION{
      VALUES ?country{wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118}
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?country .



    { ?something wdt:P170 ?writer .               
     }UNION{
      ?something wdt:P50 ?writer .
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}
      ?writer ?authorOf ?something .
    }                           

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

df5 = query_wikidata(endpoint, qf_5, user_agent, True).fillna("")
df5

,writer,writerLabel
0,http://www.wikidata.org/entity/Q1228,Oscar Luigi Scalfaro
1,http://www.wikidata.org/entity/Q1242,Luigi Einaudi
2,http://www.wikidata.org/entity/Q1233,Sandro Pertini
3,http://www.wikidata.org/entity/Q2291,Totò
4,http://www.wikidata.org/entity/Q615,Lionel Messi
...,...,...
5329,\tat org.eclipse.jetty.util.thread.strategy.Ea...,
5330,\tat org.eclipse.jetty.util.thread.ReservedThr...,
5331,\tat org.eclipse.jetty.util.thread.QueuedThrea...,
5332,\tat org.eclipse.jetty.util.thread.QueuedThrea...,


#### `qf_6` adds to the previous query the existence of an ID for that person on IMDb
- results: 913

In [61]:
qf_6 = """
SELECT DISTINCT ?writer ?writerLabel 
WHERE {
    {
      ?country wdt:P706 wd:Q145694 . #country is anything located in the Italian peninsula
    } UNION {
      ?country wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }UNION{
      VALUES ?country{wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118}
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?country .
    
    ?writer wdt:P345 ?imdbId . 


    { ?something wdt:P170 ?writer .                
     }UNION{
      ?something wdt:P50 ?writer .
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}
      ?writer ?authorOf ?something .
    }

                              

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
df6 = query_wikidata(endpoint, qf_6, user_agent, True).fillna("")
df6

,writer,writerLabel
0,http://www.wikidata.org/entity/Q2291,Totò
1,http://www.wikidata.org/entity/Q615,Lionel Messi
2,http://www.wikidata.org/entity/Q2604,Pius XII
3,http://www.wikidata.org/entity/Q2022,Cesare Pavese
4,http://www.wikidata.org/entity/Q624,Alessandro Del Piero
...,...,...
908,http://www.wikidata.org/entity/Q16941587,Colombino Arona
909,http://www.wikidata.org/entity/Q7937306,Vittorio Capellaro
910,http://www.wikidata.org/entity/Q3847140,Maria Antonietta Torriani
911,http://www.wikidata.org/entity/Q12786354,Branka Jurca


### Integrate various queries to get only one DataFrame

In [20]:
qf_7 = """
SELECT DISTINCT ?writer ?writerLabel ?imdbId (COUNT(DISTINCT ?somethingAB) AS ?countAB) (COUNT(DISTINCT ?somethingBA) AS ?countBA)
WHERE {
    {
      ?country wdt:P706 wd:Q145694 . #country is anything located in the Italian peninsula
    } UNION {
      ?country wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }UNION{
      VALUES ?country{wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118}
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?country .
    OPTIONAL { ?writer wdt:P345 ?imdbId . }        #se non lo metto in optional funziona, se lo metto in optional no

    

    { ?somethingAB wdt:P170 ?writer .                
     }UNION{
      ?somethingAB wdt:P50 ?writer .
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}
      ?writer ?authorOf ?somethingBA .
    }

 
                              
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?writer ?writerLabel ?imdbId
"""

df7 = query_wikidata(endpoint, qf_7, user_agent, True).fillna("")
df7

KeyboardInterrupt: 